### Set up the environment variables for Pyspark, Java, Spark, and python
- 오류 발생 시 

In [ ]:
import os
import sys
os.environ['JAVA_HOME'] = "C:\Java"
os.environ['SPARK_HOME'] = "C:\Bigdata\spark-3.5.1"
os.environ['PYLIB'] = "C:\Bigdata\spark-3.5.1\python\lib"
# %SPARK_HOME%\pyhon\lib 확인
sys.path.insert(0,os.environ['PYLIB']+"\py4j-0.10.9.7-src.zip")
sys.path.insert(0,os.environ['PYLIB']+"\pyspark.zip")

In [ ]:
import pyspark
import pandas as pd

#### Employee.csv
- 한글포함
- ANSI 형식으로 encoding

In [ ]:
pandasDF = pd.read_csv("Employee.csv")
#pandasDF = pd.read_csv("Employee.csv",encoding="ansi")
#pandasDF = pd.read_csv("Employee.csv",encoding="cp949")
pandasDF

In [ ]:
pandasDF.info()

### SparkSession : DataSet 및 DataFrame API를 사용하여 Spark를 프로그래밍하는 진입점
- 스파크의 기능들과 구조들이 상호방식하는 방식을 제공
- SparkContext에서 사용할 수 있는 기능은 모두 SparkSession에서 사용할 수 있음

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Test').getOrCreate()

* SparkSession.bulider : Create a basic SparkSession
* SparkSession.builder.appName() : Sets a name for the application, which will be shown in the Spark web UI
    * If no application name is set, a randomly generated name will be used
* SparkSession.builder.appName().getOrCreate() : Gets an existing SparkSession or, if there is no existing one, creates a new one based on the options set in this builder.

In [ ]:
spark

### 웹브라우저에서 localhost:4040 연결

### Pyspark에서 hdfs 데이터 불러오기
Spark 경로를 찾지 못하는 경우
```Python
!pip install findspark
import findspark
import os
findspark.find()
findspark.init(os.environ.get("SPARK_HOME"))
```

In [ ]:
sparkDF = spark.read.csv("Employee.csv")
sparkDF.show()

### Hadoop 실행 후(CMD에서 start-dfs.cmd와 start-yarn.cmd 실행/stat-all.cmd)
#### CMD> hadoop fs -mkdir /Spark
#### CMD > hadoop fs -put Employee.csv /Spark/hadoop_Employee.csv

In [ ]:
sparkDF = spark.read.csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv")
sparkDF.show()

In [ ]:
sparkDF = spark.read.option('encoding','cp949').csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv")
sparkDF.show()

In [ ]:
sparkDF = spark.read.option('encoding','cp949').option('header','true').csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv")
sparkDF.show(5)

In [ ]:
sparkDF.printSchema()

In [ ]:
data = spark.read.csv("hdfs://localhost:9000/Spark/hadoop_Employee.csv", header=True, encoding="cp949", inferSchema="true")
data.show()

In [ ]:
data.printSchema()

### DataFrame을 다른 형식으로 변환하고 저장/불러오기
- pandas DataFrame $\Longleftrightarrow$ spark DataFrame
- csv $\Longleftrightarrow$ parquet

In [ ]:
pandasDF_spark = sparkDF.toPandas()
pandasDF_spark.head()

#### pandas DataFrame을 spark DataFrame으로
Apache Spark uses Apache Arrow which is an in-memory columnar format to transfer the data between Python and JVM. 
You need to enable to use Arrow as this is disabled by default and have Apache Arrow (PyArrow) install on all Spark cluster nodes using pip install pyspark[sql] or by directly downloading from Apache Arrow for Python.

In [ ]:
sparkDF_pd  = spark.createDataFrame(pandasDF)
type(sparkDF_pd)

In [ ]:
sparkDF_pd.printSchema()

In [ ]:
sparkDF_pd.show()

### When an error occurs, 
Spark automatically fallback to non-Arrow optimization implementation, this can be controlled by spark.sql.execution.arrow.pyspark.fallback.enabled.
```pythpn
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled","true")
```

In [ ]:
### !pip install pyarrow
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")

In [ ]:
sparkDF_pd  = spark.createDataFrame(pandasDF)
sparkDF_pd.printSchema()

In [ ]:
sparkDF_pd.show()

#### Pandas에서

In [ ]:
pandasDF.to_csv('pandas_Employee.csv')
pandasDF.to_parquet('pandas_Employee.parquet')
pandasDF.to_json('pandas_Employee.json')
pandasPQ = pd.read_parquet('pandas_Employee.parquet')
pandasPQ.info()

#### Spark에서 
- sparkDF.write.csv("경로/파일이름")
- sparkDF.format('csv').save("경로/파일이름")
- 기존 파일이 있는 경우 
    - 덮어쓰기: sparkDF.write.mode('overwrite').csv("경로/파일이름")
    - 추가하기: sparkDF.write.mode('append').csv("경로/파일이름")
    - 무시하기: sparkDF.write.mode('ignore').csv("경로/파일이름")
    - 오류발생: sparkDF.write.mode('error').csv("경로/파일이름")  $\Leftarrow$ default
    

In [ ]:
sparkDF.write.csv("hdfs://localhost:9000/Spark/spark_Employee.csv")
sparkDF.write.parquet("hdfs://localhost:9000/Spark/spark_Employee.parquet")

In [ ]:
sparkPQ = spark.read.parquet("hdfs://localhost:9000/Spark/spark_Employee.parquet")
sparkPQ.show(5)

In [ ]:
sparkDF_pd = spark.read.csv("hdfs://localhost:9000/Spark/spark_Employee.csv" ,header=True)
sparkDF_pd.show(5)

In [ ]:
sparkDF.write.csv("hdfs://localhost:9000/Spark/spark_Employee.csv")

In [ ]:
sparkDF.write.mode('overwrite').csv("hdfs://localhost:9000/Spark/spark_Employee.csv",header=True)

In [ ]:
sparkDF_pd = spark.read.csv("hdfs://localhost:9000/Spark/spark_Employee.csv" ,header=True)
sparkDF_pd.show(5)

### Spark의 메모리 관리 옵션
- spark.executor.memory : Spark 애플리케이션의 각 executor 프로세스에 할당되는 메모리를 설정
- spark.executor.cores : 각 executor가 사용하는 CPU 코어 수를 설정
- spark.driver.memory : Spark 드라이버 프로세스에 할당되는 메모리를 설정
    - 드라이버는 애플리케이션의 전반적인 제어를 담당하므로 충분한 메모리를 할당, 예 '6g'
- spark.executor.instances : 클러스터에서 실행되는executor의 총 개수를 설정
    - 이 값을 조절하여 전체 병렬 처리 능력을 높이거나 낮출 수 있음
- spark.memory.fraction : Usable memory(Spark(Unified) memory+User memory)에서 Spark memory 비율(60%)
    - spark.memory.storageFraction : Spark memory(Storage memory + Execution memory)에서 Storage memory 비율 (50%)
    
예> spark.conf.set("spark.driver.memory","6g")